In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.optimizers import *
from keras.wrappers.scikit_learn import KerasRegressor
from keras.datasets import mnist
from sklearn.model_selection import GridSearchCV
import pprint
pp = pprint.PrettyPrinter(indent=4)

## Load Dataset

In [11]:
split_date='2018-07-05'
# Get normalized model_data
model_data=pd.read_csv("model_data.csv").iloc[:, 1:]

training_set, test_set = model_data[model_data['Date']<split_date], model_data[model_data['Date']>=split_date]
training_set = training_set.drop('Date', 1)
test_set = test_set.drop('Date', 1)

window_len = 35
# norm_cols = [coin+metric for coin in ['bt_'] for metric in ['Close','Volume', 'google_trends_bitcoin', 'avg_block_size', 'transactions']]

norm_cols = [coin+metric for coin in ['bt_'] for metric in ['Close','Volume', 'google_trends_bitcoin']]

# norm_cols = [coin+metric for coin in ['bt_'] for metric in ['Close','Volume']]


LSTM_training_inputs = []
for i in range(len(training_set)-window_len):
    temp_set = training_set[i:(i+window_len)].copy()
    LSTM_training_inputs.append(temp_set)   

LSTM_test_inputs = []
for i in range(len(test_set)-window_len):
    temp_set = test_set[i:(i+window_len)].copy()
    LSTM_test_inputs.append(temp_set)
    
LSTM_training_inputs = [np.array(LSTM_training_input) for LSTM_training_input in LSTM_training_inputs]
LSTM_training_inputs = np.array(LSTM_training_inputs)

LSTM_test_inputs = [np.array(LSTM_test_inputs) for LSTM_test_inputs in LSTM_test_inputs]
LSTM_test_inputs = np.array(LSTM_test_inputs)

In [13]:
# # Shape: (1689, 10, 24)
# print(LSTM_training_inputs[:-pred_range].shape)

# print(len(LSTM_training_outputs))

## Model Definition

In [21]:
import keras
# %load models/lstm_model.py
"""
    If network is overfitting => decrease batch size; the contrary is true for underfitting
"""
def lstm_model(neurons, optimizer, loss, activ_func, dropout): 
    keras.backend.clear_session()
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(100,22)))
    model.add(Dropout(dropout))
    # Units is the length of output vector, which in turn is the pred_range
    model.add(Dense(units=50))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

In [22]:
from keras import optimizers
print(optimizers.SGD)

<class 'keras.optimizers.SGD'>


## Define the Parameters

In [23]:
batch_size = [20, 50, 100][:1]
epochs = [20, 50, 100][:1]
neurons = [10, 20, 30][:1]
optimizer = ['sgd','adam'][:1]
loss=['mae','mean_squared_error']
activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'][:1]
dropout_rate = [0.3, 0.25, 0.8][:1]

## Model Wrapper and GridSearchCV

In [30]:
# Create the wrapper and pass params to GridSearchCV
params = dict(neurons = neurons,
              optimizer = optimizer,
              loss = loss,
              activ_func = activation,
              dropout = dropout_rate)

model = KerasRegressor(build_fn=lstm_model,  verbose=2, shuffle=True, batch_size = 100,
                        epochs = 100)

models = GridSearchCV(estimator = model, param_grid=params,scoring = 'neg_mean_squared_error', n_jobs=1)

In [31]:
print(len(LSTM_training_inputs[:-pred_range]))
print(len(LSTM_training_outputs))

1559
1559


## Train the Models

In [ ]:
# model output is next :pred_range prices normalised to 10th previous closing price
LSTM_training_outputs = []
for i in range(window_len, len(training_set['bt_close'])-pred_range):
    LSTM_training_outputs.append((training_set['bt_close'][i:i+pred_range].values/
                                  training_set['bt_close'].values[i-window_len])-1)
LSTM_training_outputs = np.array(LSTM_training_outputs)

best_model = models.fit(LSTM_training_inputs[:-pred_range], LSTM_training_outputs)
print('Best model :')
pp.pprint(best_model.best_params_)

Epoch 1/100
 - 1s - loss: 0.6580
Epoch 2/100
 - 0s - loss: 0.6578
Epoch 3/100
 - 0s - loss: 0.6565
Epoch 4/100
 - 0s - loss: 0.6556
Epoch 5/100
 - 0s - loss: 0.6548
Epoch 6/100
 - 1s - loss: 0.6541
Epoch 7/100
 - 0s - loss: 0.6538
Epoch 8/100
 - 0s - loss: 0.6530
Epoch 9/100
 - 1s - loss: 0.6525
Epoch 10/100
 - 1s - loss: 0.6510
Epoch 11/100
 - 1s - loss: 0.6511
Epoch 12/100
 - 0s - loss: 0.6508
Epoch 13/100
 - 0s - loss: 0.6509
Epoch 14/100
 - 1s - loss: 0.6497
Epoch 15/100
 - 1s - loss: 0.6492
Epoch 16/100
 - 0s - loss: 0.6486
Epoch 17/100
 - 1s - loss: 0.6474
Epoch 18/100
 - 1s - loss: 0.6477
Epoch 19/100
 - 1s - loss: 0.6468
Epoch 20/100
 - 1s - loss: 0.6465
Epoch 21/100
 - 1s - loss: 0.6459
Epoch 22/100
 - 1s - loss: 0.6458
Epoch 23/100
 - 1s - loss: 0.6449
Epoch 24/100
 - 1s - loss: 0.6448
Epoch 25/100
 - 1s - loss: 0.6434
Epoch 26/100
 - 2s - loss: 0.6442
Epoch 27/100
 - 1s - loss: 0.6436
Epoch 28/100
 - 1s - loss: 0.6423
Epoch 29/100
 - 1s - loss: 0.6413
Epoch 30/100
 - 1s - lo

Epoch 43/100
 - 1s - loss: 2.2502
Epoch 44/100
 - 1s - loss: 2.2511
Epoch 45/100
 - 1s - loss: 2.2502
Epoch 46/100
 - 1s - loss: 2.2506
Epoch 47/100
 - 1s - loss: 2.2504
Epoch 48/100
 - 1s - loss: 2.2508
Epoch 49/100
 - 1s - loss: 2.2503
Epoch 50/100
 - 1s - loss: 2.2502
Epoch 51/100
 - 1s - loss: 2.2503
Epoch 52/100
 - 1s - loss: 2.2497
Epoch 53/100
 - 1s - loss: 2.2503
Epoch 54/100
 - 1s - loss: 2.2500
Epoch 55/100
 - 1s - loss: 2.2498
Epoch 56/100
 - 1s - loss: 2.2503
Epoch 57/100
 - 1s - loss: 2.2500
Epoch 58/100
 - 1s - loss: 2.2503
Epoch 59/100
 - 1s - loss: 2.2497
Epoch 60/100
 - 1s - loss: 2.2496
Epoch 61/100
 - 1s - loss: 2.2491
Epoch 62/100
 - 1s - loss: 2.2500
Epoch 63/100
 - 1s - loss: 2.2501
Epoch 64/100
 - 1s - loss: 2.2494
Epoch 65/100
 - 1s - loss: 2.2492
Epoch 66/100
 - 1s - loss: 2.2497
Epoch 67/100
 - 0s - loss: 2.2496
Epoch 68/100
 - 0s - loss: 2.2495
Epoch 69/100
 - 0s - loss: 2.2496
Epoch 70/100
 - 1s - loss: 2.2500
Epoch 71/100
 - 1s - loss: 2.2496
Epoch 72/100
 

Epoch 82/100
 - 1s - loss: 95.5040
Epoch 83/100
 - 1s - loss: 94.5177
Epoch 84/100
 - 1s - loss: 94.9503
Epoch 85/100
 - 1s - loss: 94.7171
Epoch 86/100
 - 1s - loss: 94.0970
Epoch 87/100
 - 1s - loss: 93.8103
Epoch 88/100
 - 1s - loss: 96.5969
Epoch 89/100
 - 1s - loss: 95.4143
Epoch 90/100
 - 1s - loss: 95.8736
Epoch 91/100
 - 1s - loss: 95.5093
Epoch 92/100
 - 1s - loss: 95.7825
Epoch 93/100
 - 1s - loss: 95.7457
Epoch 94/100
 - 1s - loss: 95.4362
Epoch 95/100
 - 1s - loss: 95.3809
Epoch 96/100
 - 1s - loss: 95.6098
Epoch 97/100
 - 1s - loss: 95.5049
Epoch 98/100
 - 1s - loss: 95.5153
Epoch 99/100
 - 1s - loss: 95.2203
Epoch 100/100
 - 1s - loss: 94.8617
Epoch 1/100
 - 1s - loss: 100.5558
Epoch 2/100
 - 0s - loss: 100.5268
Epoch 3/100
 - 1s - loss: 100.4896
Epoch 4/100
 - 1s - loss: 100.4584
Epoch 5/100
 - 1s - loss: 100.3852
Epoch 6/100
 - 1s - loss: 100.3210
Epoch 7/100
 - 1s - loss: 100.2334
Epoch 8/100
 - 1s - loss: 100.1914
Epoch 9/100
 - 1s - loss: 100.1267
Epoch 10/100
 - 1s 